In [1]:
!pip install pytrends
!pip install fredapi
import yfinance as yf
from fredapi import Fred
import pandas as pd
from pytrends.request import TrendReq
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from functools import reduce

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
def classify_price_change(df):
    # Compare the close price with the previous day's close price
    # The result is 1 if the price is higher, 0 if lower
    # The first entry will be NaN since there is no previous day to compare with
    df['Price_Change_Class'] = (df['Close'] > df['Close'].shift(1)).astype(int)
    df['Price_Change_Class'].iloc[0] = np.nan  # Handle the first entry
    return df

In [6]:
# Initialize pytrends
#pytrends = TrendReq(hl='en-US', tz=360)

# Define the keyword and the period
#kw_list = ["Bitcoin", "Ethereum", "Cryptocurrency"]
#pytrends.build_payload(kw_list, cat=0, timeframe='today 5-y')

In [7]:
#pytrends_df = pytrends.interest_over_time()
##pytrends_df.reset_index().tail(3)
# Write pytrends_df to csv
#pytrends_df.to_csv('pytrends.csv')
# Transform

In [8]:
#pytrends_2 = TrendReq(hl='en-US', tz=360)
#pytrends_2.build_payload(kw_list, cat=0, timeframe='2014-01-01 2018-11-10')

In [9]:
#pytrends_df_2 = pytrends_2.interest_over_time()
#pytrends_df_2.reset_index().head(3)
#pytrends_df_2.to_csv('pytrends_2.csv')

In [10]:
# Combine pytrends_df and pytrends_df_2 on date
#pytrends_comb = pd.concat([pytrends_df, pytrends_df_2], ignore_index=False)

In [11]:
#pytrends_comb.tail(5)

In [12]:
fred = Fred(api_key='dabf739175778a6ed5294802c22f09f6')


In [13]:
interest_rates = fred.get_series('FEDFUNDS')
# Convert interest_rates series to df
interest_rates_df = interest_rates.to_frame().reset_index()
interest_rates_df = interest_rates_df.rename(columns={'index': 'Date', 0: 'FEDFUNDS'})
interest_rates_df['Date']

0     1954-07-01
1     1954-08-01
2     1954-09-01
3     1954-10-01
4     1954-11-01
         ...    
827   2023-06-01
828   2023-07-01
829   2023-08-01
830   2023-09-01
831   2023-10-01
Name: Date, Length: 832, dtype: datetime64[ns]

In [14]:
#Convert datetime to correct format
interest_rates_df['Date'] = pd.to_datetime(interest_rates_df['Date'])
interest_rates_df.head()

,Date,FEDFUNDS
0,1954-07-01,0.80
1,1954-08-01,1.22
2,1954-09-01,1.07
3,1954-10-01,0.85
4,1954-11-01,0.83


In [15]:
# Need to fill in the in-between on months to fill in the daily values so we can match to bitcoin eventually
interest_rates_df.set_index('Date', inplace=True)
# Resample to daily frequency and forward fill the 'FEDFUNDS' values
interest_rates_daily_df = interest_rates_df.resample('D').ffill()

# Reset the index to turn 'Date' back into a column
interest_rates_daily_df.reset_index(inplace=True)

In [16]:
interest_rates_daily_df.head()

,Date,FEDFUNDS
0,1954-07-01,0.8
1,1954-07-02,0.8
2,1954-07-03,0.8
3,1954-07-04,0.8
4,1954-07-05,0.8


In [17]:
interest_rates_daily_df.tail(3)

,Date,FEDFUNDS
25292,2023-09-29,5.33
25293,2023-09-30,5.33
25294,2023-10-01,5.33


Example of creating a feature using the simple moving average of the last three days

In [18]:
# Filters columns from history
hist_btc_df = pd.DataFrame(yf.Ticker('BTC-USD').history(period='120mo'), columns=['Close', 'Volume'])
# Moves index (date) to column
hist_btc_df.reset_index(inplace=True)
hist_btc_df['Date'] = pd.to_datetime(hist_btc_df['Date']).dt.date

In [19]:
hist_btc_df.tail()

,Date,Close,Volume
3357,2023-11-26,37479.121094,13744796068
3358,2023-11-27,37254.167969,19002925720
3359,2023-11-28,37831.085938,21696137014
3360,2023-11-29,37858.492188,20728546658
3361,2023-12-01,38179.929688,19619575808


In [20]:
# Create new column 'SMA_3' of rolling average of 'Close' column
hist_btc_df['SMA_Close_3'] = hist_btc_df['Close'].rolling(3).mean()
hist_btc_df.head(4)

,Date,Close,Volume,SMA_Close_3
0,2014-09-17,457.334015,21056800,NaN
1,2014-09-18,424.440002,34483200,NaN
2,2014-09-19,394.795990,37919700,425.523336
3,2014-09-20,408.903992,36863600,409.379995


In [21]:
hist_btc_df['SMA_Close_9'] = hist_btc_df['Close'].rolling(9).mean()
hist_btc_df.head(10)

,Date,Close,Volume,SMA_Close_3,SMA_Close_9
0,2014-09-17,457.334015,21056800,NaN,NaN
1,2014-09-18,424.440002,34483200,NaN,NaN
2,2014-09-19,394.795990,37919700,425.523336,NaN
3,2014-09-20,408.903992,36863600,409.379995,NaN
4,2014-09-21,398.821014,26580100,400.840332,NaN
5,2014-09-22,402.152008,24127600,403.292338,NaN
6,2014-09-23,435.790985,45099500,412.254669,NaN
7,2014-09-24,423.204987,30627700,420.382660,NaN
8,2014-09-25,411.574005,26814400,423.523326,417.446333
9,2014-09-26,404.424988,21460800,413.067993,411.567552


In [22]:
hist_btc_df['SMA_Close_18'] = hist_btc_df['Close'].rolling(18).mean()
hist_btc_df.head(5)

,Date,Close,Volume,SMA_Close_3,SMA_Close_9,SMA_Close_18
0,2014-09-17,457.334015,21056800,NaN,NaN,NaN
1,2014-09-18,424.440002,34483200,NaN,NaN,NaN
2,2014-09-19,394.795990,37919700,425.523336,NaN,NaN
3,2014-09-20,408.903992,36863600,409.379995,NaN,NaN
4,2014-09-21,398.821014,26580100,400.840332,NaN,NaN


In [23]:
hist_btc_df['EWA_Close_3'] = hist_btc_df['Close'].ewm(span=3, min_periods=0).mean()
hist_btc_df.head(5)

,Date,Close,Volume,SMA_Close_3,SMA_Close_9,SMA_Close_18,EWA_Close_3
0,2014-09-17,457.334015,21056800,NaN,NaN,NaN,457.334015
1,2014-09-18,424.440002,34483200,NaN,NaN,NaN,435.404673
2,2014-09-19,394.795990,37919700,425.523336,NaN,NaN,412.199711
3,2014-09-20,408.903992,36863600,409.379995,NaN,NaN,410.441994
4,2014-09-21,398.821014,26580100,400.840332,NaN,NaN,404.444069


In [24]:
hist_btc_df['EWA_Close_9'] = hist_btc_df['Close'].ewm(span=9, min_periods=9).mean()
hist_btc_df.head(5)

,Date,Close,Volume,SMA_Close_3,SMA_Close_9,SMA_Close_18,EWA_Close_3,EWA_Close_9
0,2014-09-17,457.334015,21056800,NaN,NaN,NaN,457.334015,NaN
1,2014-09-18,424.440002,34483200,NaN,NaN,NaN,435.404673,NaN
2,2014-09-19,394.795990,37919700,425.523336,NaN,NaN,412.199711,NaN
3,2014-09-20,408.903992,36863600,409.379995,NaN,NaN,410.441994,NaN
4,2014-09-21,398.821014,26580100,400.840332,NaN,NaN,404.444069,NaN


In [25]:
hist_btc_df['EWA_Close_15'] = hist_btc_df['Close'].ewm(span=15, min_periods=15).mean()
hist_btc_df.head(5)

,Date,Close,Volume,SMA_Close_3,SMA_Close_9,SMA_Close_18,EWA_Close_3,EWA_Close_9,EWA_Close_15
0,2014-09-17,457.334015,21056800,NaN,NaN,NaN,457.334015,NaN,NaN
1,2014-09-18,424.440002,34483200,NaN,NaN,NaN,435.404673,NaN,NaN
2,2014-09-19,394.795990,37919700,425.523336,NaN,NaN,412.199711,NaN,NaN
3,2014-09-20,408.903992,36863600,409.379995,NaN,NaN,410.441994,NaN,NaN
4,2014-09-21,398.821014,26580100,400.840332,NaN,NaN,404.444069,NaN,NaN


In [26]:
classified_df = classify_price_change(hist_btc_df)
classified_df.head(5)

<ipython-input-5-651c205781b5>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Price_Change_Class'].iloc[0] = np.nan  # Handle the first entry


,Date,Close,Volume,SMA_Close_3,SMA_Close_9,SMA_Close_18,EWA_Close_3,EWA_Close_9,EWA_Close_15,Price_Change_Class
0,2014-09-17,457.334015,21056800,NaN,NaN,NaN,457.334015,NaN,NaN,NaN
1,2014-09-18,424.440002,34483200,NaN,NaN,NaN,435.404673,NaN,NaN,0.0
2,2014-09-19,394.795990,37919700,425.523336,NaN,NaN,412.199711,NaN,NaN,0.0
3,2014-09-20,408.903992,36863600,409.379995,NaN,NaN,410.441994,NaN,NaN,1.0
4,2014-09-21,398.821014,26580100,400.840332,NaN,NaN,404.444069,NaN,NaN,0.0


Example of combining ethereum data onto bitcoin dataframe

In [27]:
# Filters columns from history
hist_eth_df = pd.DataFrame(yf.Ticker('ETH-USD').history(period='max'), columns=['Close', 'Volume'])
# Moves index (date) to column
hist_eth_df.reset_index(inplace=True)
# Change Volume Column to Volume_ETH
hist_eth_df = hist_eth_df.rename(columns={'Volume': 'Volume_ETH', 'Close':'Close_ETH'})
hist_eth_df['Date'] = pd.to_datetime(hist_eth_df['Date']).dt.date

In [28]:
hist_eth_df.head(5)

,Date,Close_ETH,Volume_ETH
0,2017-11-09,320.884003,893249984
1,2017-11-10,299.252991,885985984
2,2017-11-11,314.681000,842300992
3,2017-11-12,307.907990,1613479936
4,2017-11-13,316.716003,1041889984


In [29]:
usdc = pd.DataFrame(yf.Ticker('USDC-USD').history(period='max'), columns=['Close', 'Volume'])
usdc.reset_index(inplace=True)
# Change column names for understandablity
usdc = usdc.rename(columns={'Volume': 'Volume_USDC', 'Close': 'Close_USDC'})

In [30]:
usdc['Date'] = pd.to_datetime(usdc['Date']).dt.date

In [31]:
usdc.head(3)

,Date,Close_USDC,Volume_USDC
0,2018-10-08,1.00221,382900
1,2018-10-09,1.00686,108803
2,2018-10-10,1.00962,711783


In [32]:
# Combine dfs
dfList = [hist_btc_df, hist_eth_df, usdc] #pytrends_df
df = reduce(lambda x, y: x.merge(y, on='Date'), dfList)

In [33]:
df['vol_btc_3'] = df['Volume'].rolling(3).mean().shift(1)
df['vol_usdc_3'] = df['Volume_USDC'].rolling(3).mean().shift(1)
df.head(3)

,Date,Close,Volume,SMA_Close_3,SMA_Close_9,SMA_Close_18,EWA_Close_3,EWA_Close_9,EWA_Close_15,Price_Change_Class,Close_ETH,Volume_ETH,Close_USDC,Volume_USDC,vol_btc_3,vol_usdc_3
0,2018-10-08,6652.229980,3979460000,6614.496745,6590.725586,6607.989475,6624.234683,6600.126100,6591.634266,1.0,229.255005,1470740000,1.00221,382900,NaN,NaN
1,2018-10-09,6642.640137,3580810000,6632.606771,6592.623372,6602.861138,6633.437410,6608.628908,6598.010000,0.0,227.981995,1405130000,1.00686,108803,NaN,NaN
2,2018-10-10,6585.529785,3787650000,6626.799967,6592.168891,6595.280572,6609.483598,6604.009083,6596.449973,0.0,225.768997,1384040000,1.00962,711783,NaN,NaN


In [34]:
# Drop column Volume and
df = df.drop('Volume', axis=1)

In [35]:
df.to_csv('FINAL_MODEL_TRAIN.csv')